In [3]:
import torch

In [4]:
import numpy as np

In [5]:
from torchtext import data, datasets

In [6]:
import random

In [7]:
seed = 966

In [8]:
torch.manual_seed(seed)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
device

device(type='cpu')

In [11]:
TEXT = data.Field(tokenize="spacy", tokenizer_language="en_core_web_sm",lower=True)
LABEL = data.LabelField()

In [12]:
train, test = datasets.TREC.splits(TEXT, LABEL)
train, val = train.split(random_state=random.seed(seed))

In [13]:
vars(train[-1])

{'text': ['how', 'do', 'you', 'say', '2', 'in', 'latin', '?'], 'label': 'ENTY'}

In [14]:
TEXT.build_vocab(train, min_freq=2)

In [15]:
LABEL.build_vocab(train)

In [16]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, val, test), 
    batch_size = 64,
    sort_key = lambda x: len(x.text),
    device=device
)

In [17]:
import torch.nn as nn
import torch.nn.functional as F


In [18]:
class CNN(nn.Module):
    
    def __init__(
        self, vocabulary_size, embedding_size, kernels_number, kernel_sizes,
        output_size, dropout_rate):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.convolution_layers = nn.ModuleList(
            nn.ModuleList(
                [
                    nn.Conv2d(
                    in_channels=1, 
                    out_channels=kernels_number, 
                    kernel_size=(k, embedding_size))
                for k in kernel_sizes
            ]))
        self.dropout = nn.Dropout(dropout_rate)
        self.fully_connected = nn.Linear(
            len(kernel_sizes) * kernels_number, output_size)
    
    def forward(self, text):
        
        text = text.permut(1,0)
        input_embeddings = self.embedding(text)
        input_embeddings = input_embeddings.unsqueeze(1)
        conved = [
            F.relu(convolution_layer(input_embeddings)).squeeze(3)
            for convolution_layer in self.convolution_layers
        ]
        
        pooled = [
            F.max_pool1d(conv, conv.shape[2]).squeeze(3) for conv in conved
        ]
        
        concat = self.dropout(torch.cat(pooled, dim=1))
        
        final_output = self.fully_connected(concat)
        
        return final_output

In [19]:
vocabulary_size = 2679
embedding_size = 100
kernels_number = 100
kernel_sizes = [2, 3, 4]
output_size = 6
dropout_rate = 0.3


In [20]:
model = CNN(
    vocabulary_size, embedding_size, kernels_number, kernel_sizes, 
    output_size, dropout_rate)

In [21]:
print(model)

CNN(
  (embedding): Embedding(2679, 100)
  (convolution_layers): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (fully_connected): Linear(in_features=300, out_features=6, bias=True)
)


In [22]:
model.to(device)

CNN(
  (embedding): Embedding(2679, 100)
  (convolution_layers): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (fully_connected): Linear(in_features=300, out_features=6, bias=True)
)

In [23]:
import torch.optim as optim
import torch.nn as nn


In [24]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [26]:
optimizer = optim.Adam(model.parameters())

In [27]:
def accuracy(predictions, actual_label):
    max_predictions = predictions.argmax(dim=1, keepdim = True, )
    correct_predictions = max_predictions.squeeze().eq(actual_label)
    accuracy = correct_predictions.sum() / torch.cuda.FloatTensor([actual_label.shape[0]])
    return accuracy

In [28]:
def train(model, iterator, optimizer, criterion):
    model.train()
    
    epoch_loss = 0
    epoch_acc = 0 
    
    with torch.no_grad():
    
        for batch in iterator:
            optimizer.zero_grad()

            predictions = model(batch.text)

            loss = criterion(predictions, batch.label)

            acc = categorical_accuracy(predictions, batch.label)

            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()
            
            epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
    